In [1]:
from langchain_community.document_loaders import PyPDFLoader

pdf_filepath = "../05/res/차세대 한국형 스마트팜 개발.pdf"
loader = PyPDFLoader(pdf_filepath)

data = loader.load()

In [2]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
data = text_splitter.split_documents(data)

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings

import os

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=os.environ['OPENAI_API_KEY'])

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24726/95399457.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=os.environ['OPENAI_API_KEY'])


In [4]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(data, embeddings)

In [5]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.environ['OPENAI_API_KEY'])

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24726/1634795862.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.environ['OPENAI_API_KEY'])


In [6]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24726/952344495.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [7]:
from langchain.chains import ConversationalRetrievalChain

conversation_chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type='stuff',
  retriever=vectorstore.as_retriever(),
  memory=memory
)

In [8]:
query = "한국형 스마트팜이란?"
result = conversation_chain({"question": query})

answer = result['answer']

answer

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24726/4129783744.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query})


'한국형 스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등의 기술을 활용하여 농산물의 생육환경을 최적화하고 노동력을 절감하며 생산성을 향상시키는 효율적인 농업 형태를 말합니다. 한국형 스마트팜은 한국의 농업환경과 상황에 맞춘 독자적인 농업생산기술을 개발하여 국내외에서 경쟁력을 확보하는 것을 목표로 합니다. 이를 통해 농가 소득 향상과 농업기술의 국제경쟁력 향상을 도모하고 있습니다.'